In [1]:
import random
import re
import nltk
import joblib
import numpy as np
import pickle

# Function to construct markov model

In [2]:
def construct_markov_model(model,n, data):
    for item in data:
        if  len(item)<n :
            continue
        add_ngram(model, n, item)
    return model

In [5]:
#size of n gram  need to be constructed
ngram_size = 2

# Function to create n-gram

In [4]:
def add_ngram(model, n, seq):
    seq = list(seq[:]) + [None]
    for i in range(len(seq)-n):
        gram = tuple(seq[i:i+n])
        next_item = seq[i+n]
        if gram not in model:
            model[gram] = []
        model[gram].append(next_item)

# Function to generate words for suggestion

In [7]:
def generator( model,n = 2, start=None, max=1):
  try:
      if start is None :
        start = random.choice(key)
      output = list(start)
      for i in range(max):
        start = tuple(output[-n:])
        next = random.choice(model[start])
        if next is None:
          break
        else:
          output.append(next)
      return " ".join(output[n:])

  except(KeyError):
    return " ".join(start)

# Preprocessing functions

In [6]:
def sentence_split(data):

    data = re.sub(r'unk|\n \n = [^=]*[^=] = \n \n|[\/\\-]|[0-9,!?."";()-\,=<>@#%$*&~_{}:\[\]/\'\`]+', '', data)
    sentences = data.split("\n")
    new_data = []
    for sentence in sentences:
        new_data.append(sentence)
    sentences = [s.strip() for s in new_data if len(s) > 1]
    return sentences


In [7]:
def tokenizer(sentences):
    tokens = []
    for sentence in sentences:
        sentence = sentence.lower()
        token = nltk.word_tokenize(sentence)
        tokens.append(token)
    return tokens

# Consturcting Markov Chain using wikipedia dataset

In [8]:
def read_file(i):
    path = f"//home//ticvictech//article//article{i}.txt"
    with open(path , "r") as f:
        input_data = f.read()
    return input_data

In [ ]:
markov_model = {}
for i in range(40000):
    content = read_file(i)
    content = sentence_split(content)
    content = tokenizer(content)
    markov_model =  construct_markov_model(markov_model,ngram_size,content)
    content = ""

In [ ]:
import pickle
fp = open("markov_recm","wb")
pickle.dump(markov_model,fp)
fp.close()

# Constructing Markov Chain using chat datatset on top of wikipedia article dataset 

In [7]:
fp1 = open("markov_recm","rb")
recomender = pickle.load(fp1)
fp1.close()

In [8]:
import pandas as pd
df = pd.read_csv("/home/ticvictech/chat/Conversation.csv")

In [9]:
dialog_data = ""
for data in df["question"]:
    dialog_data += data+"\n"

In [11]:
processed_dialog_data = tokenizer(sentence_split(dialog_data))

In [15]:
model_2 = construct_markov_model(ngram_size,processed_dialog_data)

In [30]:
import pickle
fp = open("suggestion_model","wb")
pickle.dump(model_2,fp)
fp.close()

# Generating recommendation for given 2-gram

In [3]:
fp2 = open("text_suggestion_model","rb")
recomender = pickle.load(fp2)
fp2.close()

In [8]:
for i in range(10):
    print( i,  generator( recomender,ngram_size,max=2,start = tuple("what do ".split())))
    print( i,  generator(recomender,ngram_size, max=3,start = tuple("what do ".split())))

0 you solve
0 you think this
1 you say
1 you intend on
2 you think
2 you laws to
3 women see
3 you know working
4 you know
4 women want forced
5 you want
5 you deal with
6 we get
6 you believe that
7 media do
7 we do have
8 you have
8 you sleep waking
9 you not
9 we pass from
